# LastFM Recommender Genre Topic Modeling: Word2Vec
July [X], 2018

In [1]:
import sys
sys.executable

'/Users/brianmcmahon/anaconda3/envs/recommender/bin/python'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from os import path, getcwd

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE

from wordcloud import WordCloud, ImageColorGenerator, STOPWORDS

from PIL import Image

import gensim
from gensim import matutils, models

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

CWD = '/Volumes/BC_Clutch/Dropbox/recommender_system/'

In [3]:
# sets output to three decimals
pd.set_option('display.float_format',lambda x: '%.3f' %x)

In [4]:
tag_data = pd.read_csv('../data/recsys_tags2.csv')
tag_data = tag_data.set_index(['idx'])
tag_data['tags'] = tag_data['tags'].apply(lambda x: ', '.join(x.split('|')))
tag_data.head()

,name,tags
idx,,
0,3 Doors Down,"00s, 2008, 90s, acoustic, acoustic rock, alter..."
1,30 Seconds to Mars,"00s, <3, acoustic, addictive, alternative, alt..."
2,3OH!3,"00s, 10s, 2007, 2008, 2009, 2010, <3, addictiv..."
3,50 Cent,"00s, american, awesome, catchy, club, dance, g..."
4,A Day to Remember,"<3, acoustic, alternative, american, christian..."


In [5]:
tag_data.tags[0]

'00s, 2008, 90s, acoustic, acoustic rock, alternative, alternative rock, american, awesome, cute, grunge, guilty pleasure, hard rock, love, male vocalists, melancholic, other great tracks, pop, pop rock, post-grunge, prejebeno, punk, rock, romantic, soft rock, soundtrack'

In [6]:
tag_data.shape

(505, 2)

### Assumptions

In [7]:
data = tag_data.tags
n_topics = 6
seed = 42
ngram_range = (1,3)
ncols = 2

sw = set(STOPWORDS)

adds = ['rock','pop','hardcore','vocalist','male','female']
for word in adds:
    sw.add(word)


### Word2Vec

In [57]:
documents = tag_data.tags
texts = [[word for word in document.lower().split() if word not in sw]
         for document in documents]

In [58]:
model = gensim.models.Word2Vec(texts, size=100, window=10, min_count=1, workers=2,sg=1)

In [59]:
# list(model.wv.vocab.items())

In [60]:
len(model.wv.vocab.items())

522

In [61]:
model.most_similar('metal',topn=8)

/Users/brianmcmahon/anaconda3/envs/recommender/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('thrash', 0.9959140419960022),
 ('speed', 0.9938734769821167),
 ('symphonic', 0.9893640279769897),
 ('hard,', 0.981787383556366),
 ('technical', 0.9816805720329285),
 ('metalcore,', 0.9813909530639648),
 ('nu', 0.9808696508407593),
 ('nu-metal,', 0.9735522866249084)]

In [62]:
model.most_similar('teen',topn=8)

/Users/brianmcmahon/anaconda3/envs/recommender/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('soul,', 0.9979000091552734),
 ('summer', 0.9943442344665527),
 ('stuff,', 0.990067720413208),
 ('sexy,', 0.9883826971054077),
 ('spanish,', 0.9811021089553833),
 ('sick,', 0.9801688194274902),
 ('smooth,', 0.9773135185241699),
 ('urban', 0.9762908816337585)]

In [63]:
model.most_similar('80s',topn=8)

/Users/brianmcmahon/anaconda3/envs/recommender/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('1981,', 0.9963778853416443),
 ('1992,', 0.9962934255599976),
 ('60s,', 0.9957478046417236),
 ('1995,', 0.9953866004943848),
 ("80's,", 0.9953277111053467),
 ('aana,', 0.9941872358322144),
 ('1991,', 0.9940966963768005),
 ("90's,", 0.9939833879470825)]

In [64]:
model.most_similar('romantic',topn=8)

/Users/brianmcmahon/anaconda3/envs/recommender/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('special', 0.9943697452545166),
 ('twee', 0.9915478229522705),
 ('babe', 0.9904686212539673),
 ('edge', 0.9892690181732178),
 ('3mmey', 0.9889140725135803),
 ('spanish', 0.9872620105743408),
 ('sad', 0.9867910146713257),
 ('sexy', 0.986470103263855)]

In [65]:
model.similarity('techno','metal')

/Users/brianmcmahon/anaconda3/envs/recommender/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.5503265725244776